In [192]:
import sqlite3 #allows accessing the database using a nonstandard variant of the SQL query language
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
db = 'chinook.db'
cur = conn.cursor()

In [193]:
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn) #Opens a connection to the SQLite database file database.

Question
1. Show each table name!
2. Show each table name and keys!
3. Show Customers (just their full names, customer ID and country) who are not in the US!
4. Show yearly income varies according to genre
5. Show monthly income varies in 2013 for blues genre
6. Show only the Employees who are Sales Agents
7. Show a unique list of billing countries from the Invoice table
8. Show the invoices associated with each sales agent.
9. Show the Invoice Total, Customer name, Country and Sale Agent name for all invoices and customers.
10. Show how many Invoices in 2009 and 2011? What are the respective total sales for each of those years?
11. Show purchased track name AND artist name with each invoice line item.
12. Show the # of invoices per country.
13. Show the total number of tracks in each playlist.
14. Show all the Tracks, but displays no IDs. The resultant table should include the Album name, Media type and Genre.
15. Show total sales made by each sales agent.
16. Which sales agent made the most in sales in 2009?
17. Which sales agent made the most in sales over all?
18. Show the top 5 most purchased tracks over all
19. Show the top 3 best selling artists.
20. Show customer that has spent the most on music for each country.

In [230]:
#1. Show each table name!
def show_table():
    q1 = '''
        SELECT 
            name
        FROM 
            sqlite_master 
        WHERE 
            type ='table' AND 
            name NOT LIKE 'sqlite_%';
        '''
    return run_query(q1)
show_table()


,name
0,album
1,artist
2,customer
3,employee
4,genre
5,invoice
6,invoice_line
7,media_type
8,playlist
9,playlist_track


In [242]:
#2. Show each table name and keys!

rows = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]

def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'
def show_table_name_keys():
    for table in tables:
        print("table: " + table)
        rows = cur.execute("PRAGMA table_info({})".format(sql_identifier(table)))
        print("key_list: ")
        print(rows.fetchall())
        rows = cur.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table))) 
        print("foreign_key_list: ")
        print(rows.fetchall())
        print("\n")
        
show_table_name_keys()



table: album
key_list: 
[(0, 'album_id', 'INTEGER', 1, None, 1), (1, 'title', 'NVARCHAR(160)', 1, None, 0), (2, 'artist_id', 'INTEGER', 1, None, 0)]
foreign_key_list: 
[(0, 0, 'artist', 'artist_id', 'artist_id', 'NO ACTION', 'NO ACTION', 'NONE')]


table: artist
key_list: 
[(0, 'artist_id', 'INTEGER', 1, None, 1), (1, 'name', 'NVARCHAR(120)', 0, None, 0)]
foreign_key_list: 
[]


table: customer
key_list: 
[(0, 'customer_id', 'INTEGER', 1, None, 1), (1, 'first_name', 'NVARCHAR(40)', 1, None, 0), (2, 'last_name', 'NVARCHAR(20)', 1, None, 0), (3, 'company', 'NVARCHAR(80)', 0, None, 0), (4, 'address', 'NVARCHAR(70)', 0, None, 0), (5, 'city', 'NVARCHAR(40)', 0, None, 0), (6, 'state', 'NVARCHAR(40)', 0, None, 0), (7, 'country', 'NVARCHAR(40)', 0, None, 0), (8, 'postal_code', 'NVARCHAR(10)', 0, None, 0), (9, 'phone', 'NVARCHAR(24)', 0, None, 0), (10, 'fax', 'NVARCHAR(24)', 0, None, 0), (11, 'email', 'NVARCHAR(60)', 1, None, 0), (12, 'support_rep_id', 'INTEGER', 0, None, 0)]
foreign_key_list: 

In [255]:
#3. Show Customers (just their full names, customer ID and country) who are not in the US!
def show_customers_not_US():
    q3 = '''
        SELECT 
            first_name || " " || last_name as "Full Name" , 
            customer_id, 
            country 
        FROM customer 
        WHERE country != 'USA'
        limit 5;
        '''
    return run_query(q3)

show_customers_not_US()

,Full Name,customer_id,country
0,Luís Gonçalves,1,Brazil
1,Leonie Köhler,2,Germany
2,François Tremblay,3,Canada
3,Bjørn Hansen,4,Norway
4,František Wichterlová,5,Czech Republic


In [254]:
#4. Show yearly income varies according to genre
def show_income_genre():
    q4 = '''

SELECT
    STRFTIME('%Y', i.invoice_date) Year, 
    SUM(il.Quantity * il.unit_price) Income,
    g.Name GenreName
FROM Invoice i
    JOIN invoice_line il
    ON i.invoice_id = il.invoice_id
    JOIN Track t
    ON t.track_id = il.track_id
    JOIN Genre g
    ON t.genre_id = g.genre_id
WHERE GenreName IN ('Rock', 'Blues', 'Bossa Nova', 'Jazz', 'Latin')
GROUP BY 1,3
limit 5;
'''
    return run_query(q4)

show_income_genre()

,Year,Income,GenreName
0,2017,14.85,Blues
1,2017,34.65,Jazz
2,2017,20.79,Latin
3,2017,756.36,Rock
4,2018,43.56,Blues


In [262]:
#5. Show monthly income varies in 2017 for blues genre
def show_monthly_income_2017_blues_genre():
    q5 = '''

SELECT
    STRFTIME('%Y', i.invoice_date) Year, 
    STRFTIME('%m', i.invoice_date) Month,
    SUM(il.Quantity * il.unit_price) Income,
    g.Name GenreName
FROM Invoice i
    JOIN invoice_line il
    ON i.invoice_id = il.invoice_id
    JOIN Track t
    ON t.track_id = il.track_id
    JOIN Genre g
    ON t.genre_id = g.genre_id
WHERE GenreName = 'Blues' AND Year = '2017'
GROUP BY 1,2
limit 5;
'''
    return run_query(q5)

show_monthly_income_2017_blues_genre()

,Year,Month,Income,GenreName
0,2017,01,1.98,Blues
1,2017,02,1.98,Blues
2,2017,03,1.98,Blues
3,2017,04,0.99,Blues
4,2017,05,1.98,Blues
